In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 4 goals
1. find out country of the user
2. build a ML model to predict fraud
3. explain what variables differentiate two groups

In [2]:
country = pd.read_csv("IpAddress_to_Country.csv")
user = pd.read_csv("Fraud_Data.csv")

In [3]:
#find out min and max ip of each country
country_min_ip = country.groupby('country')['lower_bound_ip_address'].min().reset_index()
country_max_ip = country.groupby('country')['upper_bound_ip_address'].max().reset_index()
country_whole = pd.merge(country_min_ip, country_max_ip, on='country', how='inner')

In [4]:
#basic exploration of user table
#1. find out null value percentage
for col in user.columns:
    print(col, " has: {}% null values".format(user[col].isnull().sum() / user.shape[0]))

#Good to know that no column has null values, now we map id by country

user_id  has: 0.0% null values
signup_time  has: 0.0% null values
purchase_time  has: 0.0% null values
purchase_value  has: 0.0% null values
device_id  has: 0.0% null values
source  has: 0.0% null values
browser  has: 0.0% null values
sex  has: 0.0% null values
age  has: 0.0% null values
ip_address  has: 0.0% null values
class  has: 0.0% null values


In [6]:
user.head(3)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1


In [24]:
country.loc[(country.lower_bound_ip_address < 3.503114e+08) & (country.upper_bound_ip_address > 3.503114e+08), 'country'].values

array(['United States'], dtype=object)

In [25]:
user['country'] = user['ip_address'].map(lambda val: country.loc[(country.lower_bound_ip_address < val) & \
                                                    (country.upper_bound_ip_address > val), 'country'].values)

KeyboardInterrupt: 

In [15]:
user.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,"4554 Japan Name: country, dtype: object"
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,"1017 United States Name: country, dtype: ob..."
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,"43965 United States Name: country, dtype: o..."
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,"Series([], Name: country, dtype: object)"
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,"1604 United States Name: country, dtype: ob..."
